In [281]:
import sys
import pandas as pd
import numpy as np

# Define a random seed for reproducibility
seed = 0
np.random.seed(seed)

import sklearn
import xgboost
import keras
import tensorflow as tf

import os
import pickle


print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('XGBoost: {}'.format(xgboost.__version__))
print('Keras: {}'.format(keras.__version__))

Python: 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]
Pandas: 1.0.3
Numpy: 1.18.1
Sklearn: 0.22.1
XGBoost: 1.1.1
Keras: 2.3.1


In [282]:
# import the processed dataset
df = pd.read_csv('../data/processed/NSQIP_Clean2.csv')

In [283]:
from IPython.display import HTML

data = df
data = data.drop(columns=['index.1', 'index', 'Unnamed: 0']).copy()
# Replace missing values with median values
data['AGE'].replace(np.NaN, data['AGE'].median(), inplace=True)
data = data.dropna()
HTML(pd.DataFrame(data.describe()).to_html())

,READMISSION1,SMOKE,HXCOPD,HYPERMED,DISCANCR,WNDINF,STEROID,WTLOSS,EMERGNCY,RETURNOR,BLEEDIS_0.0,BLEEDIS_1.0,BLEEDIS_2.0,DIABETES,DYSPNEA,PRSEPIS,ASACLAS,SEX_female,SEX_male,RACE_American Indian or Alaska Native,RACE_Asian,RACE_Black or African American,RACE_Native Hawaiian or Pacific Islander,RACE_Unknown/Not Reported,RACE_White,HISPANIC_N,HISPANIC_U,HISPANIC_Y,TRAN_From acute care hospital inpatient,TRAN_Not transferred (admitted from home),TRAN_Nursing home - Chronic care - Intermediate care,TRAN_Outside emergency department,TRAN_Transfer from other,DISCHDEST_Home,DISCHDEST_Rehab,"DISCHDEST_Skilled Care, Not Home",DISCHDEST_Unknown,AGE,WEIGHT,PUFYEAR,OPTIME,TOTHLOS,PRSODM,PRBUN,PRCREAT,PRALBUM,PRPLATE,PRPTT,PRINR,PRPT
count,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000,1533.000000
mean,0.095238,0.132420,0.019569,0.450750,0.018917,0.001957,0.110241,0.011742,0.022831,0.043705,0.596869,0.005219,0.397913,0.183953,0.033920,0.011089,0.647913,0.497717,0.502283,0.003262,0.075669,0.139596,0.003262,0.144162,0.629485,0.427267,0.078278,0.096543,0.018917,0.954338,0.005219,0.016960,0.003262,0.941292,0.023483,0.016308,0.001957,0.026155,-0.012054,0.278520,0.040004,0.013494,0.040544,0.045120,0.049754,0.017983,0.038107,0.020548,0.058598,-0.310085
std,0.293639,0.339058,0.138561,0.497731,0.136277,0.044209,0.313293,0.107756,0.149413,0.204505,0.490687,0.072074,0.489627,0.387572,0.181083,0.104755,0.157283,0.500158,0.500158,0.057036,0.264553,0.346680,0.057036,0.351368,0.483100,0.494843,0.268696,0.295430,0.136277,0.208819,0.072074,0.129164,0.057036,0.235155,0.151482,0.126698,0.044209,0.780166,0.995876,0.587100,1.049216,0.950889,0.931535,0.943032,0.920401,1.003296,0.947537,0.994657,0.965576,0.563074
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.220814,-5.202635,-0.702101,-1.683243,-11.505392,-3.517534,-2.568833,-3.325918,-0.839085,-2.845054,-1.330860,-1.696501,-0.429450
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-0.142679,-0.610939,-0.357072,-0.613731,-0.279138,0.243796,0.234812,0.293387,-0.839085,-0.243175,-1.330860,0.586251,-0.429450
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.109216,-0.098789,0.332986,-0.215285,-0.167987,0.275672,0.337698,0.297743,-0.839085,0.146222,0.694512,0.588994,-0.429450
75%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.750000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.298137,0.484004,0.67801

In [284]:
from sklearn.model_selection import train_test_split

features = data.drop(columns=['READMISSION1'])
labels = data.READMISSION1

X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size = 300, random_state = 0)

In [285]:
from sklearn.metrics import roc_auc_score

def permutation_importance(X, y, model): 
    
    """ Add random permutations to each variable in training data to assess variable importance """
    
    perm = {}
    y_true = model.predict_proba(X)[:,1]
    baseline= roc_auc_score(y, y_true)
    
    print('Baseline ROC AUC score: {}'.format(baseline))

    for cols in X.columns:
        new_df = X.copy()
        values = new_df.loc[:,cols]
        new_df.loc[:,cols] = np.random.permutation(values)
        y_true = model.predict_proba(new_df)[:,1]
        perm[cols] = roc_auc_score(y, y_true) - baseline
        
    return perm

In [286]:
# load all optimized models from the models folder

dirName = '../models/'

fileList = list()
dirList = list()

for (dirpath, dirnames, filenames) in os. walk(dirName):
    for file in filenames:
        if '.sav' in file:
            fileList.append(os.path. join(dirpath, file))

modelList = list()

for file in fileList:
    model = pickle.load(open(file, 'rb'))
    modelList.append(model)

In [287]:
# run permutation analysis for each variable and each model

permutations = []

for i,model in enumerate(modelList):
    perms = permutation_importance(X_train.copy(), Y_train.copy(), model)
    permutations.append(perms)
    
scores = pd.DataFrame(permutations, index = [file.split('/')[-1].rstrip('.sav') for file in fileList])
scores

Baseline ROC AUC score: 0.710369524279937
Baseline ROC AUC score: 0.8878731516068287
Baseline ROC AUC score: 0.9595360108665453
Baseline ROC AUC score: 0.8211033247923936
Baseline ROC AUC score: 0.7926172321180501
Baseline ROC AUC score: 0.8548065940172259
Baseline ROC AUC score: 0.9830441762109098


,SMOKE,HXCOPD,HYPERMED,DISCANCR,WNDINF,STEROID,WTLOSS,EMERGNCY,RETURNOR,BLEEDIS_0.0,...,OPTIME,TOTHLOS,PRSODM,PRBUN,PRCREAT,PRALBUM,PRPLATE,PRPTT,PRINR,PRPT
AdaBoost,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.069540,0.000000,...,0.000000,-0.019696,0.000000,-0.065620,0.000000,0.000000,0.000000,-0.001138,0.000000,0.000000
DecisionTree,0.000000,0.000000,0.000000,0.000000,0.000000,-0.017519,0.000000,0.000000,-0.104934,0.000000,...,-0.110028,-0.033013,0.000000,-0.201382,-0.056278,-0.018333,-0.010153,-0.053711,-0.022864,0.000000
KMean,-0.000336,0.000826,0.003033,0.000413,0.000000,-0.001239,0.000000,0.000000,0.000455,0.001652,...,-0.015536,-0.001239,-0.000031,0.001972,0.000413,-0.008783,-0.003211,-0.003473,-0.002964,-0.003523
MLP,-0.047715,-0.002678,-0.034213,-0.001999,-0.003211,-0.011507,-0.000201,-0.004766,-0.094936,-0.096232,...,-0.065458,-0.011480,-0.006321,-0.027278,-0.015227,-0.080430,-0.036304,-0.112852,-0.011160,-0.010917
RandomForest,-0.005487,0.000000,-0.000054,0.000000,0.000000,-0.001351,0.000000,0.000000,-0.064088,0.000000,...,-0.006352,-0.008721,-0.006097,-0.038890,-0.005919,-0.008675,-0.012310,-0.009315,0.000085,0.000116
SVC,-0.013537,-0.000386,-0.002917,-0.002076,-0.000031,-0.002825,-0.002632,-0.001142,-0.063849,-0.014687,...,-0.035633,-0.015173,-0.015628,-0.022173,-0.012557,-0.028872,-0.016477,-0.027900,-0.022597,-0.008559
XGBoost,-0.003149,0.000000,-0.003349,0.000000,0.000000,-0.000888,0.000000,0.000000,-0.064181,-0.000478,...,-0.023053,-0.007123,-0.012642,-0.032962,-0.012418,-0.004399,-0.014324,-0.014347,-0.007015,0.000000


In [288]:
# print total permutation values for all optimized models

# Note - more negative values mean the algorithm was impacted significantly by permutations of this column (ie, this column is an important variable)
# Note - positive values may indicate the algorithm's performance is decreased by inclusion of this variable. 

totals = scores.sum(axis=0)
totals.sort_values()

RETURNOR                                               -0.461072
PRBUN                                                  -0.386333
OPTIME                                                 -0.256058
PUFYEAR                                                -0.240102
PRPTT                                                  -0.222737
PRALBUM                                                -0.149492
WEIGHT                                                 -0.129295
BLEEDIS_0.0                                            -0.109746
PRCREAT                                                -0.101986
TOTHLOS                                                -0.096444
PRPLATE                                                -0.092779
BLEEDIS_2.0                                            -0.077964
SMOKE                                                  -0.070224
PRINR                                                  -0.066515
SEX_female                                             -0.064015
AGE                      

In [289]:
# Print top 10 most important variables
totals.sort_values()[:10]

RETURNOR      -0.461072
PRBUN         -0.386333
OPTIME        -0.256058
PUFYEAR       -0.240102
PRPTT         -0.222737
PRALBUM       -0.149492
WEIGHT        -0.129295
BLEEDIS_0.0   -0.109746
PRCREAT       -0.101986
TOTHLOS       -0.096444
dtype: float64